In [1]:
import pandas as pd
from sqlalchemy import create_engine
conn1 = create_engine('impala://172.24.20.123:21050')

In [10]:
sql = '''
SELECT orders_code'订单号', od_amount'订单金额', 
    case l_status
        when 1 then '长时'
        when 2 then '短时'
    end as '类别'
FROM (
    SELECT distinct order_code, l_status
    from (
        SELECT order_code, 
            first_value(protected_type) over(PARTITION BY order_code ORDER BY last_modified_date DESC) l_status
        from jkbd.ods_customer_order_protected_parquet
        where enabled=true
    ) dup
) ocopp
right join (
    SELECT fedf.orders_code, sum(fedf.product_amount) od_amount
    from jkbd.fact_export_detail_full fedf
    -- 必备条件
    where fedf.EL_RECORD_STATE = 2 
    	and fedf.EF_RECORD_STATE = 2 
    	and fedf.order_type not in ('门店', '批零一体化', '批发')
    	and fedf.export_form_type not in ('退货单')
    	and fedf.form_rank = 1
    	and fedf.is_group = 0
    	and fedf.is_gift = '非赠品'
    	and fedf.product_code not in (857932, 857934)
    	-- 业务条件
    	and (
    	    -- 退包后
    	    fedf.order_status not in (0,5,60,80,100,160,170,180,200,305)
    	    -- 全球购不能用状态，用支付时间
    	    or (fedf.sls_platform = '全球购' and fedf.pay_time is not null
    	    	and order_status not in (200))
    	)
    	and orders_code in (
            '4519191671554069','4519192726763560','4519200854618778','4519191671174857','4519186670017078','4519200854420148','4519209697802205','5619189747994399',
            '5319188884607008','5319188884067190','5319188884983005','5319190946354054','5619190946136554','5319189747328493','5619189656497096','1120190626223014816',
            '1120190805783003200','1120190723453005553','1120190807763004420','5619188884636705','5619191671451460','5619196948164766','6819200854330751','5319203814556269',
            '6819205942638896','5619210951489278','6819191855801669','6819192726551354','5319194940700866','6819203814069466','6819207794151581','5619184587658999',
            '5319211857998793','5619192726052239','5319192726427966','5619194940005460','5619195659076869','5619195659763681','5319206977154178','5619206977910790',
            '5619206518137469','5619206518788790','5619210951676775','6819200854917190','6819200708990254','6819205942125372','6819210951176763','5619183671731775',
            '5619183671583175','5319189747704193','5619191671791269','5619191671625078','5619191671536178','6819195659853587','5319196948488287','6819197774272178',
            '5619202600064142','5619210951408475','6819213802075651','6819186921318381','8819187998790299','5619190946834369','5619201920899284','6819207794013184',
            '5319212810713163','8819212810098766','5319212810824869','8819197774836114','8819197774886914','6819210951272754','8819194940170057','5619193942703172',
            '5319196960485984','5619193829156057','5619193829089563','5319191855317969','5319192726553857','5319193829915778','5319194571911284','5619191671396472',
            '5619197729562569','5319197774930890','5319197774030463','5319198776521584','5619193829294054','5619193942427575','5619196948711675','5619194940034257',
            '5619201920306160','5619201920759681','5619201859298372','5619201920572754','5619202600924269','5619204961689699','5619204663930481','8819205611625384',
            '5619205611659187','5619205942644081','5619205942635584','5619206977109893','5619206518943781','5319207794882702','5319201920198660','5619208720535184',
            '5619208693230681','5619209697493099','5619211931440648','5619211931519878','5619212810513357','5319214668860778','5319213654150951','5619215721721169',
            '5319219758666202','5619197774261272','5619214668519196','4519198918055260','5319211931271845','5619206977205384','5619211931183748','1119198918516669',
            '5619204961171460','5619209990877123','5619213802270739','5319200854579481','5319200708195642','5619197774235284','5319207794557408','5319208693514802',
            '6819212907136854','8819214668124078','5819194940232057','4519202600035942','6819198918066869','5619214668519196','5819169840152669','8819176859929617',
            '5619205611908393','4519206977167984','6819223843906493','5619224590427096','5619224590930590','5619209697303196','5319189747749508','5819203814183348',
            '5319205611228272','5319205611514869','5619211857120445','5619222623207963','5319224590377793','5619224590666399','5619206977846805','5619206977991793',
            '5619206977708308','5619211857339778','5619214668981984','5619211931454760','5619204961238884','5619204961059281','5619204961893487','5619206977930690',
            '6819207693649620','5319207794189802','5619210951603666','5619211931120939','5319211931236860','5619211857590157','5319214668729002','5619213802118360',
            '5619225554093163','5319168710595578','5619184587863290','5619187998024280','5619191671020945','8819190712300340','5619207794048696','5619210951455166',
            '5319208693119705','5319210951843572','5319209697667617','5619208720117378','5319211931786275','5319210951361551','5319209697901108','5619179519151657',
            '5619189747290866','1119192726453842','5619191671019472','5319191855665072','5119153525610554','5319193829933672','5619204961802884','5619203814084148',
            '1119192726453842','4519183992237281','5319185899351375','5319189747057081','5619191671897490','1120190711483010965','5319192726763463','8819186670322784',
            '8819204663640281','5319197729008772','5319197729630966','5619203814702463','8819200708282945','5619207794625105','5319197774819814','5619205611128269',
            '1120190712363011422','5619206977917411','1120190718553012904','5619203814545366','1120190710273004801','8819204663640281')
    GROUP BY fedf.orders_code
) od_info on ocopp.order_code=od_info.orders_code
'''

In [11]:
df1 = pd.read_sql(sql, conn1)

In [13]:
df1.to_excel('东莞慢病_1907撞单情况.xlsx', sheet_name='info')